# 数据迭代

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_iterator.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_iterator.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/advance/dataset/iterator.ipynb)

原始数据集通过数据集加载接口读取到内存，再通过数据增强操作进行数据变换，最后得到的数据集对象有两种常规的数据迭代方法：

1. 创建`iterator`迭代器进行数据迭代。
2. 数据直接传入网络模型的Model接口（如`model.train`、`model.eval`等）进行迭代训练或推理。

## 创建迭代器

数据集对象通常可以创建两种不同的迭代器来遍历数据，分别为：

1. **元组迭代器**。创建元组迭代器的接口为`create_tuple_iterator`，通常用于`Model.train`内部使用，其迭代出来的数据可以直接用于训练。
2. **字典迭代器**。创建字典迭代器的接口为`create_dict_iterator`，自定义`train`训练模式下，用户可以根据字典中的`key`进行进一步的数据处理操作，再输入到网络中，使用较为灵活。

下面通过示例介绍两种迭代器的使用方式：

In [1]:
import mindspore.dataset as ds

# 数据集
np_data = [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]

# 加载数据
dataset = ds.NumpySlicesDataset(np_data, column_names=["data"], shuffle=False)

然后使用`create_tuple_iterator`或者`create_dict_iterator`创建数据迭代器。

In [2]:
# 创建元组迭代器
print("\n create tuple iterator")
for item in dataset.create_tuple_iterator():
    print("item:\n", item[0])

# 创建字典迭代器
print("\n create dict iterator")
for item in dataset.create_dict_iterator():
    print("item:\n", item["data"])

# 直接遍历数据集对象（等同于创建元组迭代器）
print("\n iterate dataset object directly")
for item in dataset:
    print("item:\n", item[0])

# 使用enumerate方式遍历（等同于创建元组迭代器）
print("\n iterate dataset using enumerate")
for index, item in enumerate(dataset):
    print("index: {}, item:\n {}".format(index, item[0]))


 create tuple iterator
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]

 create dict iterator
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]

 iterate dataset object directly
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]

 iterate dataset using enumerate
index: 0, item:
 [[1 2]
 [3 4]]
index: 1, item:
 [[5 6]
 [7 8]]


如果需要产生多个epoch的数据，可以相应地调整入参`num_epochs`的取值。相比于多次调用迭代器接口，直接设置epoch数可以提高数据迭代的性能。

In [3]:
# 创建元组迭代器产生2个epoch的数据
epoch = 2

iterator = dataset.create_tuple_iterator(num_epochs=epoch)

for i in range(epoch):
    print("epoch: ", i)
    for item in iterator:
        print("item:\n", item[0])

epoch:  0
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]
epoch:  1
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]


迭代器默认输出的数据类型为`mindspore.Tensor`，如果希望得到`numpy.ndarray`类型的数据，可以设置入参`output_numpy=True`。

In [4]:
# 默认输出类型为mindspore.Tensor
for item in dataset.create_tuple_iterator():
    print("dtype: ", type(item[0]), "\nitem:\n", item[0])

# 设置输出类型为numpy.ndarray
for item in dataset.create_tuple_iterator(output_numpy=True):
    print("dtype: ", type(item[0]), "\nitem:\n", item[0])

dtype:  <class 'mindspore.common.tensor.Tensor'> 
item:
 [[1 2]
 [3 4]]
dtype:  <class 'mindspore.common.tensor.Tensor'> 
item:
 [[5 6]
 [7 8]]
dtype:  <class 'numpy.ndarray'> 
item:
 [[1 2]
 [3 4]]
dtype:  <class 'numpy.ndarray'> 
item:
 [[5 6]
 [7 8]]


> 更详细的说明，请参考[create_tuple_iterator](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/dataset/mindspore.dataset.NumpySlicesDataset.html#mindspore.dataset.NumpySlicesDataset.create_tuple_iterator) 和[create_dict_iterator](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/dataset/mindspore.dataset.NumpySlicesDataset.html#mindspore.dataset.NumpySlicesDataset.create_dict_iterator)的API文档。

## 数据迭代训练

数据集对象创建后，可通过传入`Model`接口，由接口内部进行数据迭代，并送入网络执行训练或推理。实例代码如下：

In [5]:
import numpy as np
from mindspore import ms_function
from mindspore import nn, Model
import mindspore.dataset as ds
import mindspore.ops as ops

# 创建自定义数据集
def create_dataset():
    np_data = [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]
    np_data = np.array(np_data, dtype=np.float16)
    dataset = ds.NumpySlicesDataset(np_data, column_names=["data"], shuffle=False)
    return dataset

# 创建一个神经网络
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = ops.ReLU()
        self.print = ops.Print()

    @ms_function
    def construct(self, x):
        self.print(x)
        return self.relu(x)

if __name__ == "__main__":
    dataset = create_dataset()
    network = Net()
    model = Model(network)
    model.train(epoch=1, train_dataset=dataset)